In [1]:
#Load cleaned_airline_data.csv into Spark Dataframe
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ml-delay').getOrCreate()
df = spark.read.csv('cleaned_airline_data_v5.csv', header = True, inferSchema = True)

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/anderl23/hadoop-3.3.4/spark-3.3.1/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/anderl23/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


2022-12-15T03:43:42,653 WARN [main] org.apache.spark.util.Utils - Your hostname, DESKTOP-SHBG2MR resolves to a loopback address: 127.0.1.1; using 192.168.0.115 instead (on interface wifi0)
2022-12-15T03:43:42,663 WARN [main] org.apache.spark.util.Utils - Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-12-15T03:43:43,726 WARN [Thread-4] org.apache.hadoop.util.NativeCodeLoader - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df.printSchema()

root
 |-- FL_DATE: timestamp (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEPT_TIME: double (nullable = true)
 |-- DEPT_TIME: double (nullable = true)
 |-- DEP_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: double (nullable = true)
 |-- WHEELS_ON: double (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- CRS_ARR_TIME: double (nullable = true)
 |-- ARR_TIME: double (nullable = true)
 |-- ARR_DELAY: integer (nullable = true)
 |-- CANCELLED: integer (nullable = true)
 |-- DIVERTED: integer (nullable = true)
 |-- CRS_ELAPSED_TIME: integer (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- FL_YEAR: integer (nullable = true)
 |-- FL_MONTH: integer (nullable = true)
 |-- FL_DOM: integer (

In [3]:
#uncomment to calculate row count
#Show number of rows in dataframe
df.count()

35841068

In [4]:
#show first five rows in dataframe. (note this has been transposed in order to easily view)
import pandas as pd
pd.DataFrame(df.take(5), columns=df.columns).transpose()

,0,1,2,3,4
FL_DATE,2013-01-01 00:00:00,2013-01-01 00:00:00,2013-01-01 00:00:00,2013-01-01 00:00:00,2013-01-01 00:00:00
OP_CARRIER,VX,VX,VX,VX,VX
OP_CARRIER_FL_NUM,108,114,11,121,124
ORIGIN,LAX,LAX,JFK,PHL,LAX
DEST,IAD,IAD,SFO,LAX,PHL
CRS_DEPT_TIME,700.0,2205.0,730.0,700.0,1100.0
DEPT_TIME,700.0,2204.0,729.0,700.0,1104.0
DEP_DELAY,0,-1,-1,0,4
TAXI_OUT,8,12,18,14,12
WHEELS_OFF,708.0,2216.0,747.0,714.0,1116.0


In [5]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
categoricalColumns = ["OP_CARRIER","OP_CARRIER_FL_NUM","ORIGIN","DEST"]

OP_CARRIER_indexer = StringIndexer(inputCol="OP_CARRIER", outputCol="OP_CARRIER_Index")
df_1 = OP_CARRIER_indexer.fit(df).transform(df)

onehotencoder_OP_CARRIER_vector = OneHotEncoder(inputCol="OP_CARRIER_Index", outputCol="OP_CARRIER_vec")
df_2 = onehotencoder_OP_CARRIER_vector.fit(df_1).transform(df_1)

OP_CARRIER_FL_NUM_indexer = StringIndexer(inputCol="OP_CARRIER_FL_NUM", outputCol="OP_CARRIER_FL_NUM_Index")
df_3 = OP_CARRIER_FL_NUM_indexer.fit(df_2).transform(df_2)

onehotencoder_OP_CARRIER_FL_NUM_vector = OneHotEncoder(inputCol="OP_CARRIER_FL_NUM_Index", outputCol="OP_CARRIER_FL_NUM_vec")
df_4 = onehotencoder_OP_CARRIER_FL_NUM_vector.fit(df_3).transform(df_3)

ORIGIN_indexer = StringIndexer(inputCol="ORIGIN", outputCol="ORIGIN_Index")
df_5 = ORIGIN_indexer.fit(df_4).transform(df_4)

onehotencoder_ORIGIN_vector = OneHotEncoder(inputCol="ORIGIN_Index", outputCol="ORIGIN_vec")
df_6 = onehotencoder_ORIGIN_vector.fit(df_5).transform(df_5)

DEST_indexer = StringIndexer(inputCol="DEST", outputCol="DEST_Index")
df_7 = DEST_indexer.fit(df_6).transform(df_6)

onehotencoder_DEST_vector = OneHotEncoder(inputCol="DEST_Index", outputCol="DEST_vec")
df_8 = onehotencoder_DEST_vector.fit(df_7).transform(df_7)
df_8.show(3)

2022-12-15T03:47:25,628 WARN [Thread-4] org.apache.spark.sql.catalyst.util.package - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+-------------------+----------+-----------------+------+----+-------------+---------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+-------+--------+------+------+----------------+---------------+-----------------------+---------------------+------------+----------------+----------+----------------+
|            FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEPT_TIME|DEPT_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|FL_YEAR|FL_MONTH|FL_DOM|FL_DOW|OP_CARRIER_Index| OP_CARRIER_vec|OP_CARRIER_FL_NUM_Index|OP_CARRIER_FL_NUM_vec|ORIGIN_I

In [6]:
vectorAssembler = VectorAssembler(inputCols = ['OP_CARRIER_vec', 'OP_CARRIER_FL_NUM_vec', 'ORIGIN_vec','DEST_vec',
                                               'TAXI_OUT', 'TAXI_IN', 'CANCELLED','DIVERTED','CRS_ELAPSED_TIME',
                                               'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'FL_YEAR', 'FL_MONTH',
                                               'FL_DOM', 'FL_DOW'], outputCol = 'features')
v_df = vectorAssembler.transform(df_8)
v_df = v_df.select(['features', 'ARR_DELAY'])
v_df.show(3)

+--------------------+---------+
|            features|ARR_DELAY|
+--------------------+---------+
|(7924,[15,178,717...|      -27|
|(7924,[15,689,717...|       -9|
|(7924,[15,1073,71...|      -26|
+--------------------+---------+
only showing top 3 rows



In [7]:
splits = v_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [8]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='ARR_DELAY', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

2022-12-15T03:47:28,258 WARN [dag-scheduler-event-loop] org.apache.spark.scheduler.DAGScheduler - Broadcasting large task binary with size 1240.9 KiB


2022-12-15T03:50:52,958 WARN [dag-scheduler-event-loop] org.apache.spark.scheduler.DAGScheduler - Broadcasting large task binary with size 1242.1 KiB


2022-12-15T03:50:53,886 WARN [dag-scheduler-event-loop] org.apache.spark.scheduler.DAGScheduler - Broadcasting large task binary with size 1242.0 KiB


2022-12-15T03:51:32,960 WARN [Executor task launch worker for task 3.0 in stage 22.0 (TID 270)] org.apache.spark.storage.memory.MemoryStore - Not enough space to cache rdd_75_3 in memory! (computed 17.0 MiB so far)
2022-12-15T03:51:32,988 WARN [Executor task launch worker for task 3.0 in stage 22.0 (TID 270)] org.apache.spark.storage.BlockManager - Persisting block rdd_75_3 to disk instead.
2022-12-15T03:51:33,044 WARN [Executor task launch worker for task 2.0 in stage 22.0 (TID 269)] org.apache.spark.storage.memory.MemoryStore - Not enough space to cache rdd_75_2 in memory! (computed 17.0 MiB so far)
2022-12-15T03:51:33,045 WARN [Executor task launch worker for task 6.0 in stage 22.0 (TID 273)] org.apache.spark.storage.memory.MemoryStore - Not enough space to cache rdd_75_6 in memory! (computed 17.0 MiB so far)
2022-12-15T03:51:33,046 WARN [Executor task launch worker for task 6.0 in stage 22.0 (TID 273)] org.apache.spark.storage.BlockManager - Persisting block rdd_75_6 to disk instea

2022-12-15T03:51:58,846 WARN [Executor task launch worker for task 1.0 in stage 22.0 (TID 268)] org.apache.spark.storage.BlockManager - Block rdd_75_1 could not be removed as it was not found on disk or in memory
2022-12-15T03:51:59,342 ERROR [Executor task launch worker for task 1.0 in stage 22.0 (TID 268)] org.apache.spark.executor.Executor - Exception in task 1.0 in stage 22.0 (TID 268)
java.lang.OutOfMemoryError: GC overhead limit exceeded
	at java.util.LinkedList.linkLast(LinkedList.java:142) ~[?:1.8.0_352]
	at java.util.LinkedList.add(LinkedList.java:338) ~[?:1.8.0_352]
	at org.apache.spark.sql.execution.BufferedRowIterator.append(BufferedRowIterator.java:73) ~[spark-sql_2.12-3.3.1.jar:3.3.1]
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source) ~[?:?]
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43) ~[spark-sql_2.12-3.3.1.jar:3.3.1]
	at org.apache.spark.sql.execution.W

2022-12-15T03:52:03,714 WARN [task-result-getter-2] org.apache.spark.scheduler.TaskSetManager - Lost task 1.0 in stage 22.0 (TID 268) (192.168.0.115 executor driver): java.lang.OutOfMemoryError: GC overhead limit exceeded
	at java.util.LinkedList.linkLast(LinkedList.java:142)
	at java.util.LinkedList.add(LinkedList.java:338)
	at org.apache.spark.sql.execution.BufferedRowIterator.append(BufferedRowIterator.java:73)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:760)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scal

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/anderl23/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_6923/3802382033.py", line 3, in <module>
    lr_model = lr.fit(train_df)
  File "/home/anderl23/.local/lib/python3.10/site-packages/pyspark/ml/base.py", line 205, in fit
    return self._fit(dataset)
  File "/home/anderl23/.local/lib/python3.10/site-packages/pyspark/ml/wrapper.py", line 383, in _fit
    java_model = self._fit_java(dataset)
  File "/home/anderl23/.local/lib/python3.10/site-packages/pyspark/ml/wrapper.py", line 380, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "/home/anderl23/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1321, in __call__
    return_value = get_return_value(
  File "/home/anderl23/.local/lib/python3.10/site-packages/pyspark/sql/utils.py", line 190, i

ConnectionRefusedError: [Errno 111] Connection refused